In [19]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [20]:
from dotenv import load_dotenv
import os
from neo4j import GraphDatabase
import pandas as pd
from xlsx_to_csv import *

#from langchain_community.graphs import Neo4jGraph

# Warning control
import warnings
warnings.filterwarnings("ignore")

### Neo4j Credentials

In [21]:
%load_ext dotenv
%dotenv

The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [22]:
NEO4J_URI = os.getenv('NEO4J_URI')
NEO4J_USERNAME = os.getenv('NEO4J_USERNAME')
NEO4J_PASSWORD = os.getenv('NEO4J_PASSWORD')

### Connecting to Neo4j

In [23]:
URI = NEO4J_URI
AUTH = (NEO4J_USERNAME, NEO4J_PASSWORD)

In [24]:
with GraphDatabase.driver(URI, auth=AUTH) as driver:
    driver.verify_connectivity()

In [25]:
combine_data()
data = pd.read_csv('./data/combined_data.csv')
data

,name,friend,classmate,teacher,family,immediate family,lover,opponent,colleague,teammate,enemy
0,Flitwick,0,0,1,0,0,0,0,0,0,0
1,Sprout,0,0,1,0,0,0,0,0,0,0
2,Dean,1,1,0,0,0,0,0,0,0,0
3,Filch,0,0,1,1,0,0,0,0,0,0
4,Madam Pomfrey,0,0,1,0,0,0,0,0,0,0
5,Angelina Johnson,1,1,0,0,0,0,0,0,1,0
6,Charlie,0,0,0,0,0,0,0,0,0,0
7,Crabbe,0,1,0,0,0,0,0,0,0,0
8,Dudley,0,0,0,1,0,0,0,0,0,0
9,Dumbledore,0,0,1,1,0,0,0,0,0,0


### Creating Knowledge Graph

In [26]:
def create_characters(tx, name):
    tx.run("MERGE (p:Person {name: $name})", name=name)

In [27]:
def create_relationship(tx, person1, relationship, person2, attributes):
    safe_relationship = relationship.replace(" ", "_").upper()

    query = (
        f"MATCH (a:Person {{name: $person1}}), (b:Person {{name: $person2}}) "
        f"MERGE (a)-[r:{safe_relationship}]->(b) "
        "SET r += $attributes"
    )
    
    tx.run(query, person1=person1, person2=person2, attributes=attributes)


In [29]:
with driver.session() as session:
    with open('./data/combined_data.csv', 'r') as file:
        
        session.write_transaction(create_characters, 'Harry')
        
        titles = next(file)
        
        titles = titles.strip().split(',')[1:]
        
        titles = [title.upper() for title in titles]
        
        for line in file:
            
            name = line.strip().split(',')[0]

            relationships = line.strip().split(',')[1:]

            # Create nodes
            session.write_transaction(create_characters, name)

            # Create relationships
            relations = dict(zip(titles, relationships))
            
            existing_relations = [key for key, value in relations.items() if int(value) == 1]
            
            for existing_relation in existing_relations:
                session.write_transaction(create_relationship, 'Harry', existing_relation, name, {})
                session.write_transaction(create_relationship, name, existing_relation, 'Harry', {})

driver.close()